### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### Part I: How To Find The Most Popular Movies?

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

Before you implement your code for `popular_recommendations` function, we will provide a helper function, called `create_ranked_df`. This helper function transforms `movies` and `reviews` dataframes into a `ranked_movies` dataframe of movies that are sorted by the highest average rating & time and have more than 4 ratings.

In [4]:
# This helper function transforms `movies` and `reviews` dataframes
# into a `ranked_movies` dataframe of movies that are sorted 
# by the highest average rating & time and have more than 4 ratings.

def create_ranked_df(movies, reviews):
        '''
        INPUT
        movies - the movies dataframe
        reviews - the reviews dataframe
        
        OUTPUT
        ranked_movies - a dataframe with movies that are sorted by highest avg rating, more reviews, 
                        then time, and must have more than 4 ratings
        '''
        
        # Pull the average ratings and number of ratings for each movie
        movie_ratings = reviews.groupby('movie_id')['rating']
        avg_ratings = movie_ratings.mean()
        num_ratings = movie_ratings.count()
        last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
        last_rating.columns = ['last_rating']

        # Add Dates
        rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
        rating_count_df = rating_count_df.join(last_rating)

        # merge with the movies dataset
        movie_recs = movies.set_index('movie_id').join(rating_count_df)

        # sort by top avg rating and number of ratings
        ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

        # for edge cases - subset the movie list to those with only 5 or more reviews
        ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
        
        return ranked_movies

ranked_movies = create_ranked_df(movies, reviews)

In [5]:
ranked_movies.head()

,movie,genre,date,1800's,1900's,2000's,Documentary,Adventure,Animation,Thriller,...,Film-Noir,Sport,Mystery,Western,Horror,Comedy,Music,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
5512872,Be Somebody (2016),Comedy|Drama|Romance,2016,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,10.000000,41.0,2018-05-17 00:29:18
59113,Doctor Zhivago (1965),Drama|Romance|War,1965,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,5.0,2017-11-19 09:22:58
986264,Taare Zameen Par (2007),Drama|Family,2007,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,9.666667,18.0,2020-03-23 20:57:38
9095324,Coldplay: A Head Full of Dreams (2018),Documentary|Music,2018,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,9.555556,9.0,2020-02-08 16:48:18
21749,City Lights (1931),Comedy|Drama|Romance,1931,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,9.500000,10.0,2019-05-31 02:44:26


In [6]:
def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a dataframe from 

    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''

    # Implement your code here
    top_movies = list(ranked_movies.movie[:n_top])

    return top_movies

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [9]:
# Top 20 movies recommended for id 1
recs_20_for_1 = popular_recommendations(user_id=1, n_top=20, ranked_movies=ranked_movies)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(user_id=53968, n_top=5, ranked_movies=ranked_movies)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(user_id=70000, n_top=100, ranked_movies=ranked_movies)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(user_id=43, n_top=35, ranked_movies=ranked_movies)

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [10]:
### You Should Not Need To Modify Anything In This Cell

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

In [32]:
ranked_movies['date'].isin([2016, 1999]).values.shape

(3434,)

In [68]:
(ranked_movies['Animation']).values == 1
np.logical_and((ranked_movies['Animation']).values == 1, (ranked_movies['Mystery']).values == 1).sum()

14

In [63]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    
    # Implement your code here

    # Step 1: filter movies based on year and genre
    #top_movies = list(ranked_movies.movie[:n_top])
    movies_years = np.ones((ranked_movies.shape[0],))
    if years:
        years = [int(y) for y in years]
        movies_years = ranked_movies['date'].isin(years).values
    movies_genres = np.ones((ranked_movies.shape[0],))
    if genres:
        #if len(genres) > 1:
        #    for g in range(1,len(genres)):
        #        #movies_genres = np.logical_and(movies_genres, (ranked_movies[genres[g]] == 1).values)
        #        movies_genres = np.logical_or(movies_genres, (ranked_movies[genres[g]] == 1).values)
        num_genre_match = ranked_movies[genres].sum(axis=1)
        movies_genres = (num_genre_match > 0).values
                
    # Step 2: create top movies list 
    top_movies = list(ranked_movies[np.logical_and(movies_years, movies_genres)].movie[:n_top])

    return top_movies

In [64]:
# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
recs_20_for_1_filtered = popular_recs_filtered(user_id=1, n_top=20, ranked_movies=ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

# Top 5 movies recommended for id 53968 with no genre filter but years=['2015', '2016', '2017', '2018']
recs_5_for_53968_filtered = popular_recs_filtered(user_id=53968, n_top=5, ranked_movies=ranked_movies, years=['2015', '2016', '2017', '2018'], genres=None)

# Top 100 movies recommended for id 70000 with no year filter but genres=['History', 'News']
recs_100_for_70000_filtered = popular_recs_filtered(user_id=70000, n_top=100, ranked_movies=ranked_movies, years=None, genres=['History', 'News'])

In [65]:
recs_20_for_1_filtered

['Taeksi woonjunsa (2017)',
 'Ayla: The Daughter of War (2017)',
 'Hacksaw Ridge (2016)',
 'They Shall Not Grow Old (2018)',
 'Straight Outta Compton (2015)',
 'Hidden Figures (2016)',
 '13th (2016)',
 'Little Boy (2015)',
 'Under sandet (2015)',
 'Hotel Mumbai (2018)',
 'Darkest Hour (2017)',
 'Kono sekai no katasumi ni (2016)',
 'Bridge of Spies (2015)',
 'Woman in Gold (2015)',
 'The Birth of a Nation (2016)',
 'The Big Short (2015)',
 'Dunkirk (2017)',
 'Victoria &amp; Abdul (2017)',
 'Anthropoid (2016)',
 'Truth (2015)']

In [66]:
### You Should Not Need To Modify Anything In This Cell

# check 1 
assert t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History']) == recs_20_for_1_filtered,  "The first check failed..."
# check 2
assert t.popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018']) == recs_5_for_53968_filtered,  "The second check failed..."
# check 3
assert t.popular_recs_filtered('70000', 100, ranked_movies, genres=['History', 'News']) == recs_100_for_70000_filtered,  "The third check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

AssertionError: The first check failed...